In [14]:
# === Cell 1: Imports + Config ===
import logging
from pathlib import Path
from typing import Dict, List, Tuple

import joblib
import numpy as np
import pandas as pd

from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score, f1_score, recall_score, roc_auc_score, confusion_matrix
from sklearn.preprocessing import PowerTransformer
from sklearn.feature_selection import VarianceThreshold
from sklearn.pipeline import Pipeline as SkPipeline

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE

# -------------------------
# Config
# -------------------------
DATA_DIR = Path("datasets")
MODULE = "BBB"
PRESENTATIONS = ["2013B", "2013J"]

# Sinh nhiều "mốc quan sát" để tạo dataset dạng time-slicing
CUTOFFS = [3, 5, 7, 10, 14, 21, 30, 45, 60, 90, 120, 150, 180]

WINDOW_DAYS = 14       # cửa sổ feature nhìn lại 14 ngày
HALF_WINDOW = 7        # chia 14 ngày thành 2 nửa: 0-7 và 8-14
HORIZON = 14           # dự đoán vắng trong 14 ngày tương lai (HƯỚNG 1)
VAR_THRESH = 0.0

MODEL_PATH = "short_term_inactive_next14days.pkl"

logging.basicConfig(level=logging.INFO, format="[%(asctime)s] %(levelname)s - %(message)s")


In [15]:
# === Cell 2: Load raw OULAD ===
def load_raw(data_dir: Path) -> Dict[str, pd.DataFrame]:
    return {
        "student_info": pd.read_csv(data_dir / "studentInfo.csv"),
        "student_reg": pd.read_csv(data_dir / "studentRegistration.csv"),
        "student_vle": pd.read_csv(data_dir / "studentVle.csv"),
    }


def prepare_students(raw: Dict[str, pd.DataFrame], module: str, presentations: List[str]):
    reg_mod = raw["student_reg"][
        (raw["student_reg"]["code_module"] == module)
        & (raw["student_reg"]["code_presentation"].isin(presentations))
    ].copy()

    reg_lookup = reg_mod[["id_student", "date_registration"]].drop_duplicates()

    students = raw["student_info"][
        (raw["student_info"]["code_module"] == module)
        & (raw["student_info"]["code_presentation"].isin(presentations))
        & (raw["student_info"]["id_student"].isin(reg_lookup["id_student"]))
    ].copy()

    vle_mod = raw["student_vle"][
        (raw["student_vle"]["code_module"] == module)
        & (raw["student_vle"]["code_presentation"].isin(presentations))
    ].merge(reg_lookup, on="id_student", how="inner")

    # relative day since registration (handles late registration)
    vle_mod["days_since_reg"] = vle_mod["date"] - vle_mod["date_registration"]

    # keep valid relative days
    vle_mod = vle_mod[vle_mod["days_since_reg"].notna()].copy()
    vle_mod = vle_mod[vle_mod["days_since_reg"] >= 0].copy()

    logging.info("So hoc vien hop le: %d", students["id_student"].nunique())
    logging.info("So ban ghi VLE: %d", len(vle_mod))
    return students, vle_mod



In [16]:
# === Cell 3: Helpers ===
def compute_inactivity_streak(days_list: List[int], start_day: int, end_day: int) -> int:
    if not days_list:
        return end_day - start_day + 1
    active = set(days_list)
    streak, d = 0, end_day
    while d >= start_day and d not in active:
        streak += 1
        d -= 1
    return streak


def build_short_term_label(vle_mod: pd.DataFrame, cutoff: int, horizon: int = HORIZON) -> pd.DataFrame:
    """
    y_short = 1 if NO activity in (cutoff+1 .. cutoff+horizon)
    """
    future = vle_mod[(vle_mod["days_since_reg"] > cutoff) & (vle_mod["days_since_reg"] <= cutoff + horizon)]
    has_future = future.groupby("id_student").size().gt(0).astype(int).reset_index(name="has_future_activity")
    has_future["y_short"] = (has_future["has_future_activity"] == 0).astype(int)  # always 0 here; we use fillna(1) later
    return has_future[["id_student", "y_short"]]


In [17]:
# === Cell 4: Build features + label ===
def build_features_short_term(
    students: pd.DataFrame,
    vle_mod: pd.DataFrame,
    cutoffs: List[int],
    window_days: int = WINDOW_DAYS,
    half_window: int = HALF_WINDOW,
    horizon: int = HORIZON,
) -> Tuple[pd.DataFrame, List[str]]:

    student_ids = students["id_student"].unique()
    augmented = []

    for cutoff in cutoffs:
        w_start = max(0, cutoff - (window_days - 1))
        w_end = cutoff

        # logs up to cutoff
        vle_cum = vle_mod[vle_mod["days_since_reg"] <= cutoff].copy()
        vle_win = vle_cum[vle_cum["days_since_reg"] >= w_start].copy()

        # ---- base + label (IMPORTANT: do NOT filter out inactive students) ----
        base = pd.DataFrame({"id_student": student_ids})
        base["days_elapsed_since_reg"] = cutoff  # so it becomes a feature too

        label_df = build_short_term_label(vle_mod, cutoff, horizon=horizon)
        merged = base.merge(label_df, on="id_student", how="left")
        # If student not in label_df => NO activity in future => y_short=1 (inactive)
        merged["y_short"] = merged["y_short"].fillna(1).astype(int)

        # ---- cumulative agg ----
        cum_agg = (
            vle_cum.groupby("id_student")
            .agg(
                total_clicks=("sum_click", "sum"),
                active_days_total=("days_since_reg", "nunique"),
                last_active=("days_since_reg", "max"),
            )
            .reset_index()
        )
        cum_agg["clicks_per_day_total"] = cum_agg["total_clicks"] / max(cutoff, 1)
        cum_agg["active_ratio_total"] = cum_agg["active_days_total"] / max(cutoff, 1)
        cum_agg["days_since_last_active"] = cutoff - cum_agg["last_active"]
        cum_agg["avg_clicks_per_active_day_total"] = (
            cum_agg["total_clicks"] / cum_agg["active_days_total"].replace(0, np.nan)
        ).fillna(0)

        # ---- window 14 agg ----
        win_agg = (
            vle_win.groupby("id_student")
            .agg(
                clicks_last_14_days=("sum_click", "sum"),
                active_days_14=("days_since_reg", "nunique")
            )
            .reset_index()
        )
        win_agg["clicks_per_day_14"] = win_agg["clicks_last_14_days"] / window_days
        win_agg["active_ratio_14"] = win_agg["active_days_14"] / window_days

        # split 14-day window
        first_end = min(w_end, w_start + (half_window - 1))
        second_start = min(w_end, first_end + 1)

        clicks_0_7 = (
            vle_win[(vle_win["days_since_reg"] >= w_start) & (vle_win["days_since_reg"] <= first_end)]
            .groupby("id_student")["sum_click"].sum().reset_index(name="clicks_0_7")
        )
        clicks_8_14 = (
            vle_win[(vle_win["days_since_reg"] >= second_start) & (vle_win["days_since_reg"] <= w_end)]
            .groupby("id_student")["sum_click"].sum().reset_index(name="clicks_8_14")
        )

        clicks_last_7 = (
            vle_cum[vle_cum["days_since_reg"] > (cutoff - 7)]
            .groupby("id_student")["sum_click"].sum().reset_index(name="clicks_last_7_days")
        )

        # inactivity streak in last 14 days
        days_list = (
            vle_win.groupby("id_student")["days_since_reg"]
            .apply(lambda x: sorted(x.unique()))
            .reset_index()
            .rename(columns={"days_since_reg": "active_days_list"})
        )
        days_list["inactivity_streak_14"] = days_list["active_days_list"].apply(
            lambda lst: compute_inactivity_streak(lst, w_start, w_end)
        )
        streak = days_list[["id_student", "inactivity_streak_14"]]

        # ---- merge features ----
        merged = merged.merge(cum_agg, on="id_student", how="left")
        merged = merged.merge(win_agg, on="id_student", how="left")
        merged = merged.merge(clicks_0_7, on="id_student", how="left")
        merged = merged.merge(clicks_8_14, on="id_student", how="left")
        merged = merged.merge(clicks_last_7, on="id_student", how="left")
        merged = merged.merge(streak, on="id_student", how="left")

        # fill missing -> 0
        fill0 = [
            "total_clicks",
            "active_days_total",
            "last_active",
            "clicks_per_day_total",
            "active_ratio_total",
            "days_since_last_active",
            "avg_clicks_per_active_day_total",
            "clicks_last_14_days",
            "active_days_14",
            "clicks_per_day_14",
            "active_ratio_14",
            "clicks_last_7_days",
            "clicks_0_7",
            "clicks_8_14",
            "inactivity_streak_14",
        ]
        for col in fill0:
            if col in merged.columns:
                merged[col] = merged[col].fillna(0)

        merged["trend_click_14"] = merged["clicks_8_14"] - merged["clicks_0_7"]
        merged["ratio_click_14"] = (merged["clicks_8_14"] + 1) / (merged["clicks_0_7"] + 1)

        augmented.append(merged)

    final_df = pd.concat(augmented, ignore_index=True)

    feature_cols = [
        "days_elapsed_since_reg",
        "clicks_per_day_total",
        "active_ratio_total",
        "avg_clicks_per_active_day_total",
        "days_since_last_active",
        "clicks_last_14_days",
        "active_days_14",
        "clicks_per_day_14",
        "active_ratio_14",
        "clicks_last_7_days",
        "clicks_0_7",
        "clicks_8_14",
        "trend_click_14",
        "ratio_click_14",
        "inactivity_streak_14",
    ]
    return final_df, feature_cols



In [18]:
# === Cell 5: Pipelines + Models ===
def make_eval_pipe(model):
    return ImbPipeline([
        ("variance_threshold", VarianceThreshold(VAR_THRESH)),
        ("smote", SMOTE()),
        ("power_transformer", PowerTransformer()),
        ("classifier", model),
    ])

def make_prod_pipe(model):
    return SkPipeline([
        ("variance_threshold", VarianceThreshold(VAR_THRESH)),
        ("power_transformer", PowerTransformer()),
        ("classifier", model),
    ])

MODELS = {
    "LogisticRegression": LogisticRegression(
        penalty="l2",
        solver="lbfgs",
        max_iter=2000,
        class_weight="balanced",
        random_state=42
    ),
    "RandomForest": RandomForestClassifier(
        n_estimators=400,
        max_depth=14,
        min_samples_split=10,
        min_samples_leaf=4,
        random_state=42,
        n_jobs=-1,
        class_weight="balanced"
    ),
    "GradientBoosting": GradientBoostingClassifier(
        learning_rate=0.05,
        max_depth=3,
        min_samples_leaf=30,
        min_samples_split=20,
        n_estimators=200,
        random_state=42
    ),
    "MLP": MLPClassifier(
        hidden_layer_sizes=(128,),
        max_iter=1200,
        early_stopping=True,
        random_state=42
    )
}


In [19]:
# === Cell 6: Train/Eval GroupKFold + Save best ===
raw = load_raw(DATA_DIR)
students, vle_mod = prepare_students(raw, MODULE, PRESENTATIONS)

final_df, feature_cols = build_features_short_term(
    students, vle_mod, CUTOFFS,
    window_days=WINDOW_DAYS, half_window=HALF_WINDOW, horizon=HORIZON
)

X = final_df[feature_cols].fillna(0)
y = final_df["y_short"].astype(int)     # 1 = vắng 14 ngày tới
groups = final_df["id_student"]

logging.info("Train samples: %d", len(final_df))
logging.info("Positive rate (vang_14days): %.2f%%", 100 * y.mean())

gkf = GroupKFold(n_splits=5)
summary_rows = []

for name, model in MODELS.items():
    fold_rows = []
    for tr_idx, te_idx in gkf.split(X, y, groups):
        X_tr, X_te = X.iloc[tr_idx], X.iloc[te_idx]
        y_tr, y_te = y.iloc[tr_idx], y.iloc[te_idx]

        pipe = make_eval_pipe(model)
        pipe.fit(X_tr, y_tr)

        y_pred = pipe.predict(X_te)
        if hasattr(pipe, "predict_proba"):
            y_proba = pipe.predict_proba(X_te)[:, 1]
            auc = roc_auc_score(y_te, y_proba)
        else:
            auc = np.nan

        tn, fp, fn, tp = confusion_matrix(y_te, y_pred).ravel()
        fold_rows.append({
            "model": name,
            "accuracy": accuracy_score(y_te, y_pred),
            "f1": f1_score(y_te, y_pred),
            "recall_pos(vang)": recall_score(y_te, y_pred),
            "specificity": tn / (tn + fp + 1e-9),
            "auc": auc,
        })

    df = pd.DataFrame(fold_rows)
    summary_rows.append({
        "model": name,
        "mean_accuracy": df["accuracy"].mean(),
        "mean_f1": df["f1"].mean(),
        "mean_recall_pos(vang)": df["recall_pos(vang)"].mean(),
        "mean_specificity": df["specificity"].mean(),
        "mean_auc": df["auc"].mean(),
    })

summary_df = pd.DataFrame(summary_rows).sort_values("mean_f1", ascending=False)
print("=== Hiệu năng trung bình (sort theo F1) ===")
display(summary_df)

best_model_name = summary_df.iloc[0]["model"]
best_model = MODELS[best_model_name]

prod_pipe = make_prod_pipe(best_model)
prod_pipe.fit(X, y)

joblib.dump({"pipeline": prod_pipe, "feature_cols": feature_cols}, MODEL_PATH)
print(f"✅ Saved model to: {MODEL_PATH} (best={best_model_name})")


[2025-12-28 02:17:24,825] INFO - So hoc vien hop le: 3960
[2025-12-28 02:17:24,840] INFO - So ban ghi VLE: 864034
[2025-12-28 02:17:25,920] INFO - Train samples: 51480
[2025-12-28 02:17:25,920] INFO - Positive rate (vang_14days): 63.21%
/home/trong-viet/Desktop/lms/engtastic_ai/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(
/home/trong-viet/Desktop/lms/engtastic_ai/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalt

=== Hiệu năng trung bình (sort theo F1) ===


,model,mean_accuracy,mean_f1,mean_recall_pos(vang),mean_specificity,mean_auc
1,RandomForest,0.849903,0.886685,0.929256,0.713508,0.861538
0,LogisticRegression,0.846911,0.885287,0.934803,0.695797,0.835115
2,GradientBoosting,0.848524,0.885129,0.923474,0.719677,0.862372
3,MLP,0.845901,0.883982,0.928982,0.703107,0.852471


✅ Saved model to: short_term_inactive_next14days.pkl (best=RandomForest)


In [25]:
# === Cell 7: Inference (nhập feature JSON) ===
bundle = joblib.load(MODEL_PATH)
loaded_pipe = bundle["pipeline"]
feature_cols = bundle["feature_cols"]

# Case cực đoan: 14 ngày gần nhất không có hoạt động
student_data = {
    "student_id": "1001",
    "days_elapsed_since_reg": 40,

    "clicks_per_day_total": 0.05,
    "active_ratio_total": 0.05,
    "avg_clicks_per_active_day_total": 1.0,

    "days_since_last_active": 9,
    "clicks_last_14_days": 2,
    "active_days_14": 1,
    "clicks_per_day_14": 2/14,
    "active_ratio_14": 1/14,

    "clicks_last_7_days": 0,
    "clicks_0_7": 0,
    "clicks_8_14": 2,
    "trend_click_14": 2,
    "ratio_click_14": 3.0,

    "inactivity_streak_14": 8
}



input_df = pd.DataFrame([student_data])[feature_cols].fillna(0)

proba_vang = loaded_pipe.predict_proba(input_df)[0, 1]
pred = loaded_pipe.predict(input_df)[0]

print(f"--- Dự đoán NGẮN HẠN (vắng >14 ngày tới) cho student_id={student_data['student_id']} ---")
print(f"P(vắng 14 ngày tới): {proba_vang:.2%}")
print(f"Kết luận: {'⚠️ NGUY CƠ VẮNG' if pred==1 else '✅ CÓ KHẢ NĂNG VẪN HỌC'}")


--- Dự đoán NGẮN HẠN (vắng >14 ngày tới) cho student_id=1001 ---
P(vắng 14 ngày tới): 37.98%
Kết luận: ✅ CÓ KHẢ NĂNG VẪN HỌC
